In [301]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster import hierarchy
import tensorflow as tf
from sklearn.model_selection import train_test_split
%matplotlib widget
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid, softmax

In [302]:
df = pd.read_csv('Train_Data_Final.csv')

In [304]:
df1 = df[df['AH'] >= 0]
df1 = df1.drop(['Date', 'Time','id','NMHC(GT)','PT08.S2(NMHC)'], axis=1)

In [305]:
df1.columns

Index(['CO(GT)', 'PT08.S1(CO)', 'C6H6(GT)', 'NOx(GT)', 'PT08.S3(NOx)',
       'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'AH', 'RH_type'],
      dtype='object')

In [306]:
df_ = df1.replace(-200, np.nan)

In [307]:
df_ = df_.drop(['RH_type'], axis = 1)

In [308]:
imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df_)
df_imputed = pd.DataFrame(df_imputed, columns=df_.columns)
df_ = df_imputed

In [329]:
# df_normalized = (df_-df_.min())/(df_.max()-df_.min())
# df_normalized = (df_-df_.mean())/(df_.std())
df_normalized = df_/(df_.max())


In [330]:
df_ = df_normalized
df_normalized.describe()

,CO(GT),PT08.S1(CO),C6H6(GT),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,AH
count,7.711000e+03,7.711000e+03,7.711000e+03,7711.000000,7711.000000,7.711000e+03,7.711000e+03,7.711000e+03,7.711000e+03,7.711000e+03
mean,-9.214664e-18,4.975919e-17,6.450265e-18,0.000000,0.000000,-3.317279e-17,2.948692e-17,-3.317279e-17,2.948692e-17,3.685866e-17
std,1.459095e-01,2.335762e-01,1.422553e-01,0.159088,0.141557,1.872034e-01,2.614311e-01,2.628505e-01,3.448203e-01,3.435513e-01
min,-2.030793e-01,-4.788254e-01,-1.934338e-01,-0.173844,-0.287177,-4.373439e-01,-6.705951e-01,-5.147112e-01,-7.618928e-01,-7.685327e-01
25%,-1.113190e-01,-1.768653e-01,-1.078028e-01,-0.108436,-0.096615,-1.373538e-01,-1.593414e-01,-1.912859e-01,-2.545154e-01,-2.310955e-01
50%,-3.791077e-02,-3.969123e-02,-3.437230e-02,-0.051452,-0.016874,-1.561870e-02,7.581631e-04,-4.149950e-02,-7.790746e-03,-3.079815e-02
75%,7.220158e-02,1.437681e-01,7.399823e-02,0.051070,0.074045,1.093772e-01,1.578016e-01,1.653048e-01,2.347887e-01,2.518397e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [331]:
# # Calculate the mean and standard deviation for each column
# mean_values = df_normalized.mean()
# std_values = df_normalized.std()

# # Create histograms for mean and standard deviation
# fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# # Histogram for mean values
# axes[0].bar(df_normalized.columns, mean_values)
# axes[0].set_title('Histogram of Mean Values')
# axes[0].set_xlabel('Columns')
# axes[0].set_ylabel('Mean')

# # Histogram for standard deviation values
# axes[1].bar(df_normalized.columns, std_values)
# axes[1].set_title('Histogram of Standard Deviation Values')
# axes[1].set_xlabel('Columns')
# axes[1].set_ylabel('Standard Deviation')

# plt.tight_layout()
# plt.show()


In [332]:
X_train, X_test, y_train, y_test = train_test_split(df_, df1['RH_type'], train_size=0.8)
y_train.replace({"Dry": 0, "Ideal": 1, "Slightly Elevated": 2, "Elevated": 3, "High": 4}, inplace=True)
y_train = np.array(y_train).reshape(-1, 1) 
y_test.replace({"Dry": 0, "Ideal": 1, "Slightly Elevated": 2, "Elevated": 3, "High": 4}, inplace=True)
y_test = np.array(y_test).reshape(-1, 1) 
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [333]:
model = Sequential(
    [             
        Dense(units = 50, activation = 'relu'),
        Dense(units = 5, activation = 'linear')
    ], name = "my_model" 
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
)

history = model.fit(
    X_train,y_train,
    epochs=500
)

Epoch 1/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 456us/step - loss: 1.0626 
Epoch 2/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - loss: 0.4048
Epoch 3/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - loss: 0.2940
Epoch 4/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - loss: 0.2309
Epoch 5/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - loss: 0.2065
Epoch 6/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - loss: 0.1781
Epoch 7/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - loss: 0.1589
Epoch 8/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - loss: 0.1586
Epoch 9/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: 0.1374
Epoch 10/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - loss: 0.1378
Epoch 11/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - loss: 0.1214
Epoch 12/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - loss: 0.1184
Epoch 13/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - loss: 0.1092
Epoch 14/500
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - loss: 0.1112
Epoch 15/500
1

In [334]:
prediction = model.predict(X_test)
y_hat = np.array(prediction.argmax(axis=1))
y_hat = np.array(y_hat).reshape(-1,1)
acc1 = 0
for i in range(y_hat.shape[0]):
    if(y_hat[i] == y_test[i]):
        acc1+=1
acc1/=y_hat.shape[0]
acc1

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step


0.9786130913804277

In [335]:
prediction1 = model.predict(X_train)
y_hat1 = np.array(prediction1.argmax(axis=1))
y_hat1 = np.array(y_hat1).reshape(-1,1)
acc2 = 0
for i in range(y_train.size):
    if(y_train[i] == y_hat1[i]):
        acc2+=1
acc2/=y_train.shape[0]
acc2

193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step


0.9865434500648509

In [336]:
from sklearn.svm import SVC

# Create an SVM model
svm_model = SVC()

# Train the model on X_train
svm_model.fit(X_train, y_train)


c:\Users\rouna\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [337]:
prediction2 = svm_model.predict(X_test)
y_hat2 = np.array(prediction2).reshape(-1,1)
acc3 = 0
for i in range(y_hat2.shape[0]):
    if(y_hat2[i] == y_test[i]):
        acc3+=1
acc3/=y_hat2.shape[0]
acc3

0.9397278029812054

In [338]:
prediction3 = svm_model.predict(X_train)
y_hat3 = np.array(prediction3).reshape(-1,1)
acc4 = 0
for i in range(y_train.shape[0]):
    if(y_train[i] == y_hat3[i]):
        acc4+=1
acc4/=y_train.shape[0]
acc4

0.9468223086900129